# Cross validation - window size 11
(reps are averaged)

Olga'straining a

In [7]:
import pandas as pd
from Bio import SeqIO
import numpy as np
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.model_selection import train_test_split
import pickle
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## Data prep

### Table with full nucleotide sequences (+ OH)

In [8]:
seqs = SeqIO.parse(open('Homo_sapiens.GRCh38.98.transcript.fa'),'fasta')
s=[]
for fasta in seqs:
    ns = (fasta.id, str(fasta.seq))
    s.append(ns)
    
df = pd.DataFrame(s, columns = ['id', 'seq'])

one_hot = []
nuc_to_oh = {'A': [1,0,0,0,0], 'T':[0,1,0,0,0], 'C':[0,0,1,0,0], 'G':[0,0,0,1,0], 'N':[0,0,0,0,1]}
for s in df.seq:
    oh = []
    for n in s:
        oh.append(nuc_to_oh[n])
    one_hot.append(oh)

df['oh'] = np.array(one_hot, dtype=object)
df_id = df.set_index('id')
df_id['lengths'] = [len(df.seq[i]) for i in range(len(df.seq))]
df_id

,seq,oh,lengths
id,,,
ENST00000456328,GTTAACTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCAT...,"[[0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0...",1657
ENST00000450305,GTGTCTGACTTCCAGCAACTGCTGGCCTGTGCCAGGGTGCAAGCTG...,"[[0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1...",632
ENST00000488147,ATGGGAGCCGTGTGCACGTCGGGAGCTCGGAGTGAGCGCACCATGA...,"[[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1...",1351
ENST00000619216,TGTGGGAGAGGAACATGGGCTCAGGACAGCGGGTGTCAGCTTGCCT...,"[[0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0...",68
ENST00000473358,GTGCACACGGCTCCCATGCGTTGTCTTCCGAGCGTCAGGCCGCCCC...,"[[0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1...",712
...,...,...,...
ENST00000420810,ATGGCTGATTCTGAGAAAGGCAAGAAAATTTTTCTTCAGATGTGTG...,"[[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1...",319
ENST00000456738,ATGAAGTCCAATCCCCCTATCCAGGCTGCCATTGACCTCATGGCAG...,"[[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1...",342
ENST00000435945,GTCATTGTATTTCAGACATACACAAATAAAAGTCATGTGCCCATTG...,"[[0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0...",2337


In [9]:
seqs = SeqIO.parse(open('Homo_sapiens.GRCh38.98.transcript.fa'),'fasta')
s=[]
for fasta in seqs:
    ns = (fasta.id, str(fasta.seq))
    s.append(ns)
    
df = pd.DataFrame(s, columns = ['id', 'seq'])
one_hot = []
nuc_to_oh = {'A': [1,0,0,0,0], 'T':[0,1,0,0,0], 'C':[0,0,1,0,0], 'G':[0,0,0,1,0], 'N':[0,0,0,0,1]}
for s in df.seq:
    oh = []
    for n in s:
        oh.append(nuc_to_oh[n])
    one_hot.append(oh)

df['oh'] = np.array(one_hot, dtype=object)
df_id = df.set_index('id')
df_id['lengths'] = [len(df.seq[i]) for i in range(len(df.seq))]
df_id

,seq,oh,lengths
id,,,
ENST00000456328,GTTAACTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCAT...,"[[0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0...",1657
ENST00000450305,GTGTCTGACTTCCAGCAACTGCTGGCCTGTGCCAGGGTGCAAGCTG...,"[[0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1...",632
ENST00000488147,ATGGGAGCCGTGTGCACGTCGGGAGCTCGGAGTGAGCGCACCATGA...,"[[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1...",1351
ENST00000619216,TGTGGGAGAGGAACATGGGCTCAGGACAGCGGGTGTCAGCTTGCCT...,"[[0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 1, 0, 0...",68
ENST00000473358,GTGCACACGGCTCCCATGCGTTGTCTTCCGAGCGTCAGGCCGCCCC...,"[[0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1...",712
...,...,...,...
ENST00000420810,ATGGCTGATTCTGAGAAAGGCAAGAAAATTTTTCTTCAGATGTGTG...,"[[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1...",319
ENST00000456738,ATGAAGTCCAATCCCCCTATCCAGGCTGCCATTGACCTCATGGCAG...,"[[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1...",342
ENST00000435945,GTCATTGTATTTCAGACATACACAAATAAAAGTCATGTGCCCATTG...,"[[0, 0, 0, 1, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0...",2337


### Table with length-11 segments of sequences and SHAPE data

In [10]:
dataset = pd.read_csv('ids_for_shape_nucl_win11_noreps.csv').drop(['Unnamed: 0'], axis=1)
seqs = np.load('oh_nucl_11_noreps.npy').astype('uint8')
shape = np.load('shape_11_noreps.npy').astype('float16')
dataset['seq'] = list(seqs)
dataset['shape'] = list(shape)
dataset['length'] = [df_id['lengths'][dataset['id'][i]] for i in range(dataset.shape[0])]
dataset

,id,seq,shape,length
0,ENST00000000412,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[0.065, 0.156, 0.1205, 0.236, 0.2305, 0.2325, ...",2450
1,ENST00000000412,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[0.055, 0.053, 0.0115, 0.305, 0.4084, 0.2795, ...",2450
2,ENST00000000412,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[0.0, 0.0, 0.0, 0.543, 0.004, 0.07, 0.145, 0.0...",2450
3,ENST00000000412,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[0.048, 0.047, 0.093, 0.094, 0.021, 0.134, 0.0...",2450
4,ENST00000000412,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[0.1395, 0.088, 0.0455, 0.1065, 0.027, 0.2015,...",2450
...,...,...,...,...
55278144,ENST00000673027,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, ...","[0.219, 0.119, 0.1, 0.348, 0.51, 0.13, 0.921, ...",1188
55278145,ENST00000673027,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, ...","[0.123, 0.082, 0.034, 0.258, 0.0475, 0.181, 0....",1188
55278146,ENST00000673027,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, ...","[0.277, 0.0, 0.133, 0.548, 0.349, 0.241, 0.16,...",1188
55278147,ENST00000673027,"[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, ...","[0.069, 0.0, 0.2686, 0.7334, 0.256, 0.474, 0.8...",1188


In [11]:
dataset1 = dataset.iloc[:10000000]
dataset1

,id,seq,shape,length
0,ENST00000000412,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[0.065, 0.156, 0.1205, 0.236, 0.2305, 0.2325, ...",2450
1,ENST00000000412,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[0.055, 0.053, 0.0115, 0.305, 0.4084, 0.2795, ...",2450
2,ENST00000000412,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[0.0, 0.0, 0.0, 0.543, 0.004, 0.07, 0.145, 0.0...",2450
3,ENST00000000412,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[0.048, 0.047, 0.093, 0.094, 0.021, 0.134, 0.0...",2450
4,ENST00000000412,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[0.1395, 0.088, 0.0455, 0.1065, 0.027, 0.2015,...",2450
...,...,...,...,...
9999995,ENST00000307892,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0.3525, 0.0965, 0.201, 0.605, 0.1755, 0.3145,...",4286
9999996,ENST00000307892,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0.343, 0.3806, 0.121, 0.56, 0.126, 0.146, 0.1...",4286
9999997,ENST00000307892,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0.858, 0.5327, 0.1335, 1.0, 0.437, 0.092, 0.3...",4286
9999998,ENST00000307892,"[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0.6426, 0.598, 0.1755, 0.788, 0.2255, 0.082, ...",4286


In [ ]:
dataset.to_csv('dataset.csv')

In [ ]:
dataset['seq'][0]

### Splitting into train, val, test

In [ ]:
dataset1['id'].nunique()

In [ ]:
n_test_ind = int(dataset['id'].nunique()*0.2)
np.random.seed(10)
random_id_indices = np.random.choice(dataset['id'].nunique(), n_test_ind, replace=False)
ids_for_test = dataset['id'].unique()[random_id_indices]
ids_for_val = ids_for_test[:n_test_ind//2]
ids_for_test = ids_for_test[n_test_ind//2:]

In [ ]:
n_test_ind = int(dataset1['id'].nunique()*0.2)
np.random.seed(10)
random_id_indices = np.random.choice(dataset1['id'].nunique(), n_test_ind, replace=False)
ids_for_test = dataset1['id'].unique()[random_id_indices]
ids_for_val = ids_for_test[:n_test_ind//2]
ids_for_test = ids_for_test[n_test_ind//2:]

In [ ]:
n_test_ind, len(ids_for_test), len(ids_for_val)

In [ ]:
ids_for_test, ids_for_val

In [ ]:
dataset['id']

In [ ]:
test_indices = []
train_indices = []
val_indices = []
for i in range(dataset1.shape[0]):
    if dataset1['id'][i] in ids_for_test:
        test_indices.append(i)
    elif dataset1['id'][i] in ids_for_val:
        val_indices.append(i)
    else:
        train_indices.append(i)

In [ ]:
val_indices

In [ ]:
len(train_indices)
len(val_indices)

In [ ]:
np.save("Models/train_indices.npy", np.array(train_indices))
np.save("Models/test_indices.npy", np.array(test_indices))
np.save("Models/val_indices.npy", np.array(val_indices))

In [ ]:
dataset1

In [ ]:
train_set = (dataset1.iloc[train_indices]).sample(frac=1, random_state=10)
val_set = dataset1.iloc[val_indices]
test_set = dataset1.iloc[test_indices]


In [ ]:
train_set['seq']

In [ ]:
X_train = np.stack(train_set['seq'])

In [ ]:
X_train

In [ ]:
X_train = np.stack(train_set['seq'])
y_train = np.stack(train_set['shape'])
X_val = np.stack(val_set['seq'])
y_val = np.stack(val_set['shape'])
X_test = np.stack(test_set['seq'])
y_test = np.stack(test_set['shape'])
train_lengths = np.array(train_set['length'])
val_lengths = np.array(val_set['length'])
test_lengths = np.array(test_set['length'])

np.save("CrossVal_Models_pt5/X_train_11_noreps.npy", np.array(X_train))
np.save("CrossVal_Models_pt5/y_train_11_noreps.npy", np.array(y_train))
np.save("CrossVal_Models_pt5/X_val_11_noreps.npy", np.array(X_val))
np.save("CrossVal_Models_pt5/y_val_11_noreps.npy", np.array(y_val))
np.save("CrossVal_Models_pt5/X_test_11_noreps.npy", np.array(X_test))
np.save("CrossVal_Models_pt5/y_test_11_noreps.npy", np.array(y_test))
np.save("CrossVal_Models_pt5/train_lengths.npy", np.array(train_set['length']))
np.save("CrossVal_Models_pt5/val_lengths.npy", np.array(val_set['length']))
np.save("CrossVal_Models_pt5/test_lengths.npy", np.array(test_set['length']))

In [ ]:
X_train = np.load("CrossVal_Models_pt5/X_train_11_noreps.npy")
y_train = np.load("CrossVal_Models_pt5/y_train_11_noreps.npy")
X_val = np.load("CrossVal_Models_pt5/X_val_11_noreps.npy")
y_val = np.load("CrossVal_Models_pt5/y_val_11_noreps.npy")
X_test = np.load("CrossVal_Models_pt5/X_test_11_noreps.npy")
y_test = np.load("CrossVal_Models_pt5/y_test_11_noreps.npy")

I'm only using the middle SHAPE value as the target, but want to save the entire corresponding SHAPE sequences just in case. (Predictions of other positions in the future, comparing SHAPEs of different cell lines/subtypes...)

In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

In [ ]:
y_train_old = y_train
y_val_old = y_val
y_test_old = y_test

y_train = y_train[:,5]
y_val = y_val[:,5]
y_test = y_test[:,5]

In [ ]:
len(X_train[0])

In the next cell, I used test set instead of val, so now val is test :)

## Cross validation

In [ ]:
best_error = 1.0
best_model = None
for learning_rate in [0.01, 0.03, 0.1, 0.005]:
    for subsample in [0.5, 0.7, 0.9]:
        for colsample_bytree in [0.9,0.6]:
                for colsample_bylevel in [0.9,0.6]:
                    for max_depth in [20,30,45]: 
                            if learning_rate==0.01 and subsample==.5 and (colsample_bytree==.9 or (colsample_bylevel==.6 and max_depth<45)):
                                continue
                            modelname = 'CrossVal_Models_pt5/model_dep'+str(max_depth)+'_lr'+str(learning_rate)+'_subs'+str(subsample)+'_cst'+str(colsample_bytree)+'_csl'+str(colsample_bylevel)+'.sav'
                            n_estimators = 750
                            if learning_rate == 0.005:
                                n_estimators=1000
                            elif max_depth <=30 or learning_rate==0.1:
                                n_estimators = 600
                            model = XGBRegressor(max_depth=max_depth,  n_estimators=n_estimators, subsample=subsample, learning_rate=learning_rate, colsample_bytree=colsample_bytree, colsample_bylevel=colsample_bylevel, verbosity=1, n_jobs=64)
                            model.fit(X_train, y_train, eval_metric='mae', eval_set=[(X_train, y_train), (X_test,y_test)])
                            error = MAE(model.predict(X_test), y_test)
                            if error<best_error:
                                best_error=error
                                best_model=model
                            pickle.dump(model, open(modelname, 'wb'))
pickle.dump(best_model, open("CrossVal_Models_pt5/best_model.sav", 'wb'))